In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import urllib.request

%matplotlib inline



In [2]:
QAP_INSTANCE_URL = 'https://qaplib.mgi.polymtl.ca/data.d/nug12.dat'
qap_instance_file = urllib.request.urlopen(QAP_INSTANCE_URL)

line = qap_instance_file.readline()
n = int(line.decode()[:-1].split()[0])
print('Problem size: %d' % n)

A = np.empty((n, n))
qap_instance_file.readline()
for i in range(n):
    line = qap_instance_file.readline()
    A[i, :] = list(map(int, line.decode()[:-1].split()))
print('Flow matrix:\n', A)

B = np.empty((n, n))
qap_instance_file.readline()
for i in range(n):
    line = qap_instance_file.readline()
    B[i, :] = list(map(int, line.decode()[:-1].split()))
print('Distance matrix:\n', B)

p = [11, 6, 8, 2, 3, 7, 10, 0, 4, 5, 9, 1]

Problem size: 12
Flow matrix:
 [[0. 1. 2. 3. 1. 2. 3. 4. 2. 3. 4. 5.]
 [1. 0. 1. 2. 2. 1. 2. 3. 3. 2. 3. 4.]
 [2. 1. 0. 1. 3. 2. 1. 2. 4. 3. 2. 3.]
 [3. 2. 1. 0. 4. 3. 2. 1. 5. 4. 3. 2.]
 [1. 2. 3. 4. 0. 1. 2. 3. 1. 2. 3. 4.]
 [2. 1. 2. 3. 1. 0. 1. 2. 2. 1. 2. 3.]
 [3. 2. 1. 2. 2. 1. 0. 1. 3. 2. 1. 2.]
 [4. 3. 2. 1. 3. 2. 1. 0. 4. 3. 2. 1.]
 [2. 3. 4. 5. 1. 2. 3. 4. 0. 1. 2. 3.]
 [3. 2. 3. 4. 2. 1. 2. 3. 1. 0. 1. 2.]
 [4. 3. 2. 3. 3. 2. 1. 2. 2. 1. 0. 1.]
 [5. 4. 3. 2. 4. 3. 2. 1. 3. 2. 1. 0.]]
Distance matrix:
 [[ 0.  5.  2.  4.  1.  0.  0.  6.  2.  1.  1.  1.]
 [ 5.  0.  3.  0.  2.  2.  2.  0.  4.  5.  0.  0.]
 [ 2.  3.  0.  0.  0.  0.  0.  5.  5.  2.  2.  2.]
 [ 4.  0.  0.  0.  5.  2.  2. 10.  0.  0.  5.  5.]
 [ 1.  2.  0.  5.  0. 10.  0.  0.  0.  5.  1.  1.]
 [ 0.  2.  0.  2. 10.  0.  5.  1.  1.  5.  4.  0.]
 [ 0.  2.  0.  2.  0.  5.  0. 10.  5.  2.  3.  3.]
 [ 6.  0.  5. 10.  0.  1. 10.  0.  0.  0.  5.  0.]
 [ 2.  4.  5.  0.  0.  1.  5.  0.  0.  0. 10. 10.]
 [ 1.  5.  2.  0.  5.  

In [4]:
rng = np.random.default_rng()

class Annealing_Experiments:
    def __init__(self,n,A,B,p):
        self.n=n
        self.A=A
        self.B=B
        self.p=p
        self.radius=1

    def qap_objective_function(self):
        s = 0.0
        for i in range(self.n):
            s += (self.A[i, :] * self.B[self.p[i], self.p]).sum()
        return s
    
    def random_neighbor(self):
        q = self.p.copy()
        for r in range(self.radius):
            i, j = rng.choice(self.n, 2, replace=False)
            q[i], q[j] = q[j], q[i]
        return q
    
    def anneal(self):
        T = 500000
        alpha = 1.0
        t0 = time.time()

        p = np.random.permutation(n)
        p_cost = self.qap_objective_function()
        costs = np.zeros(T)
        for t in range(T):
            q = self.random_neighbor()
            q_cost = self.qap_objective_function(q)
            if(q_cost < p_cost):
                
                p, p_cost = q, q_cost
            elif(np.random.rand() < np.exp(- alpha * (q_cost - p_cost) * t/T)):
                p, p_cost = q, q_cost
            costs[t] = p_cost
        return costs
    
    def anneal_with_tracking(self):
        T = 500000
        alpha = 1.0
        t0 = time.time()

        p = np.random.permutation(n)
        p_cost = self.qap_objective_function()
        costs = np.zeros(T)
        successes = np.zeros(T)
        acc_failures = np.zeros(T)
        for t in range(T):
            q = self.random_neighbor()
            q_cost = self.qap_objective_function(q)
            if(q_cost < p_cost):
                successes[t]=1
                p, p_cost = q, q_cost
            elif(np.random.rand() < np.exp(- alpha * (q_cost - p_cost) * t/T)):
                p, p_cost = q, q_cost
                acc_failures[t]=1
            costs[t] = p_cost
        return costs,successes,acc_failures

    def run_experiment(self):
        total=np.zeros(100)
        for k in range(100):
            total[k]=self.anneal().min()
            print(k)
        return total

In [ ]:
experiment=Annealing_Experiments(n,A,B,p)